## Estadística descriptiva

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import jarque_bera
import cvxpy as cp
from cvxopt import matrix, solvers
import requests
import json
import urllib3
urllib3.disable_warnings()
from datetime import datetime
import pytz

Realizaré un análisis estadístico descriptivo para entender la distribución y las características fundamentales de los datos. Este análisis incluirá la media, mediana, desviación estándar, así como los valores mínimos y máximos de los precios.

Finalmente, crearé un gráfico de líneas que permitirá visualizar la evolución de los precios a lo largo del tiempo. Este gráfico servirá para detectar tendencias, patrones, y posibles outliers en los datos, proporcionando una base sólida para los análisis predictivos y la optimización de estrategias que se realizarán en etapas posteriores del proyecto.

In [ ]:
# Estadísticas descriptivas
descriptive_stats = datos.describe()

# Visualización inicial - Gráfico de líneas para todas las acciones
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 8))
for column in datos.columns:
    plt.plot(datos.index, datos[column], label=column)

plt.title('Precios de Acciones del Panel Líder del Merval')
plt.xlabel('Índice Temporal')
plt.ylabel('Precio')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(True)
plt.show()

descriptive_stats

### Cálculo de Retornos y Medidas de Tendencia Central

Comprender la dinámica del retorno esperado y la volatilidad es fundamental para tomar decisiones informadas.

Utilizando técnicas de estadística descriptiva, podemos cuantificar estos elementos y examinar la distribución de los retornos de los activos.

Primero, calculamos los retornos diarios de cada activo como la variación porcentual en el precio de cierre de un día para el siguiente. Esto nos proporciona una medida básica de rendimiento:

In [ ]:
df1_return = datos / datos.shift(-1) - 1

df1_return = df1_return.dropna()  # Elimina la primera fila que contiene valores nulos

df1_return

#### Retorno Esperado y Volatilidad

El retorno esperado ($\mu$)y la volatilidad ($\sigma$) son dos de las estadísticas más cruciales para la evaluación de activos.

El retorno esperado representa el promedio de los retornos pasados, mientras que la volatilidad mide la desviación estándar de esos retornos, proporcionando una medida de riesgo:

In [ ]:
# Retorno Esperado
mu = df1_return.mean()

# Volatilidad
sigma = df1_return.std()

# Crear un DataFrame combinando mu y sigma
tabla1 = pd.concat([mu, sigma], axis=1)
tabla1.columns = ['Retornos Medios', 'Volatilidad']

# Mostrar el DataFrame resultante
tabla1

#### Medidas de Dispersión y Forma

Además de las medidas de tendencia central, es importante evaluar la dispersión y la asimetría en los retornos de los activos para comprender mejor los riesgos asociados:

* **Cuartiles y Rango Intercuartil:** Los cuartiles dividen los datos en cuatro partes iguales y el rango intercuartil (la diferencia entre el tercer cuartil (Q3) y el primer cuartil (Q1)) ayuda a entender la variabilidad de los retornos excluyendo los valores atípicos.

* **Mediana de las Desviaciones Absolutas (MAD)**: Ofrece una medida robusta de la variabilidad centrada en la mediana.

* **Coeficiente de Asimetría**: Esta medida indica cómo los retornos se desvían de una distribución normal. Un valor de cero indica una distribución simétrica, mientras que valores negativos o positivos indican asimetría hacia la izquierda o derecha, respectivamente.

In [ ]:
# Calcular cuartiles para cada activo individualmente
Q1 = df1_return.quantile(0.25)
Q2 = df1_return.median()
Q3 = df1_return.quantile(0.75)

# Calcular el rango intercuartil
rango_intercuartil = Q3 - Q1

# Calcular la Mediana de las Desviaciones Absolutas (MAD)
mad = np.mean(np.abs(df1_return - df1_return.median()))

# Calcular el Coeficiente de Asimetría (Skewness)
skewness = 3 * (mu - df1_return.median()) / sigma

# Crear un DataFrame combinando mu, sigma, cuartiles, rango intercuartil, MAD y skewness
tabla1 = pd.DataFrame({
    'Retornos Medios': mu,
    'Volatilidad': sigma,
    'Q1': Q1,
    'Q2 (Mediana)': Q2,
    'Q3': Q3,
    'Rango Intercuartil': rango_intercuartil,
    'MAD': mad,
    'Coeficiente de Asimetría': skewness
})

# Mostrar el DataFrame resultante
tabla1

#### Visualización de Datos

Un gráfico de caja (boxplot) es extremadamente útil para visualizar la distribución de los retornos, mostrando visualmente la mediana, cuartiles y valores atípicos de cada activo:

In [ ]:
# Crea un gráfico de caja para cada columna en el DataFrame
plt.figure(figsize=(12, 8))  # Ajusta el tamaño del gráfico según necesites
sns.boxplot(data=df1_return)
plt.title('Boxplot para cada activo')
plt.xlabel('Activos')
plt.ylabel('Retornos')
plt.xticks(rotation=45)  # Rota las etiquetas del eje x si es necesario
plt.show()

#### Matriz de Covarianza ($\Sigma$)

La matriz de covarianza es una herramienta esencial en la teoría de portafolios para analizar cómo los retornos de diferentes activos se mueven juntos.

En términos financieros, la covarianza entre dos activos indica el grado en que estos activos se mueven en relación uno con el otro.

Un valor positivo indica que los activos tienden a moverse en la misma dirección, mientras que un valor negativo sugiere movimientos opuestos.

Aquí utilizamos la función .cov() de pandas para calcular la matriz de covarianza de los retornos:

$$ Cov (X, Y) = \frac{1}{N} \sum_{i=1}^N (x_i - \bar{x})(y_i - \bar{y})$$

$$ \bar{x} = \frac{1}{N} \sum_{i=1}^N x_i \quad\quad\quad\quad\bar{y} = \frac{1}{N} \sum_{i=1}^N y_i $$

In [ ]:
Sigma = df1_return.cov()
Sigma

#### Matriz de Correlación ($\rho_{XY}$)

Mientras que la matriz de covarianza puede ser difícil de interpretar directamente debido a la dependencia de las magnitudes de los datos, la matriz de correlación estandariza estos valores, permitiendo una comparación directa entre variables.

La correlación mide la relación lineal entre dos variables en una escala de -1 a 1. Utilizamos la función .corr() para obtener la matriz de correlación:


$$\large\rho (x,y) = \frac{1}{N} \sum_{i=1}^N \frac{(x_i - \bar{x})}{S_x} \frac{(y_i - \bar{y})}{S_y}$$

$$\large cov(x,y) = \rho (x,y) S_x S_y$$

Recordar:

$$S_x = \sqrt{\frac{1}{N} \sum_{i=1}^N (x_i - \bar{x})^2}\quad\quad S_y = \sqrt{\frac{1}{N} \sum_{i=1}^N (y_i - \bar{y})^2}$$

In [ ]:
correlation_matrix = df1_return.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='YlGnBu', fmt=".2f")
plt.title('Matriz de Correlación de Retornos (Calculada Manualmente)')

plt.show()

#### Visualización de Datos y Clasificación de Activos

Para visualizar las relaciones entre la volatilidad y los retornos medios de los activos, empleamos un gráfico de dispersión.

Este gráfico ayuda a identificar visualmente activos con características de riesgo y retorno deseables.

Los activos pueden ser categorizados basándose en su posición en el gráfico, lo que facilita decisiones estratégicas de inversión.

In [ ]:
# Configurar el tamaño de la figura
plt.figure(figsize=(12, 8))

# Crear un conjunto de colores para asignar a cada activo
colores = np.arange(len(tabla1))

# Crear un gráfico de dispersión con colores diferentes para cada activo
scatter = plt.scatter(tabla1['Volatilidad'], tabla1['Retornos Medios'], c=colores, cmap='YlGnBu', alpha=0.9)

# Etiquetar cada punto con el nombre del activo
for i, txt in enumerate(tabla1.index):
    plt.annotate(txt, (tabla1['Volatilidad'][i], tabla1['Retornos Medios'][i]), fontsize=8)

# Configurar etiquetas y título del gráfico
plt.xlabel('Volatilidad')
plt.ylabel('Retornos Medios')
plt.title('Clasificación de Activos por Retorno Medio y Volatilidad')

# Mostrar el gráfico
plt.grid(True)
plt.show()